In [4]:
from datasets import load_dataset
from sklearn.metrics import v_measure_score

dataset = load_dataset("tattabio/e_coli_rnas")  # returns a Hugging Face dataset
print(dataset["train"][0])

{'Sequence': 'GGCGGCCGTAGCGCGGTGGTCCCACCTGACCCCATGCCGAACTCAGAAGTGAAACGCCGTAGCGCCGATGGTAGTGTGGGGTCTCCCCATGCGAGAGTAG', 'Label': 'rRNA'}


In [ ]:
from datasets import load_dataset

dataset = load_dataset("tattabio/e_coli_rnas")["train"]

with open("ecoli_rna.fasta", "w") as f:
    for sample in dataset:
        seq_id = sample["id"]
        sequence = sample["sequence"].replace("U", "T")  # Convert RNA to DNA
        f.write(f">{seq_id}\n{sequence}\n")

KeyError: 'id'

In [6]:
import subprocess
from collections import defaultdict
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import v_measure_score

def save_fasta(dataset, output_path):
    with open(output_path, "w") as f:
        for i, sample in enumerate(dataset):
            seq_id = f"seq{i}"
            sequence = sample["Sequence"].replace("U", "T")  # Just in case
            f.write(f">{seq_id}\n{sequence}\n")

def run_cd_hit_est(input_fasta, output_fasta, identity=0.9, threads=4, memory=8000):
    subprocess.run([
        "cd-hit-est",
        "-i", input_fasta,
        "-o", output_fasta,
        "-c", str(identity),
        "-n", "8",
        "-T", str(threads),
        "-M", str(memory)
    ], check=True)

def parse_clstr_file(clstr_path):
    clusters = defaultdict(list)
    current_cluster = None
    with open(clstr_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">Cluster"):
                current_cluster = line.split()[-1]
            else:
                seq_id = line.split(">")[1].split("...")[0]
                clusters[current_cluster].append(seq_id)
    return clusters

def compute_v_score(clusters, labels):
    # Flatten clusters
    cluster_assignments = []
    for cluster_id, members in clusters.items():
        for seq_id in members:
            cluster_assignments.append((seq_id, cluster_id))
    cluster_df = pd.DataFrame(cluster_assignments, columns=["id", "cluster_id"])

    # Create label dataframe
    label_df = pd.DataFrame([
        (f"seq{i}", sample["Label"]) for i, sample in enumerate(labels)
    ], columns=["id", "true_label"])

    # Merge
    merged = pd.merge(cluster_df, label_df, on="id")
    v = v_measure_score(merged["true_label"], merged["cluster_id"])
    print(f"✅ V-score: {v:.4f}")
    return v

In [8]:
dataset = load_dataset("tattabio/e_coli_rnas")["train"]

fasta_file = "ecoli_rna.fasta"
clustered_file = "ecoli_rna_clustered.fa"
clstr_file = clustered_file + ".clstr"

# Step 1: Save sequences to FASTA
save_fasta(dataset, fasta_file)

# Step 2: Run CD-HIT
run_cd_hit_est(fasta_file, clustered_file, identity=0.9)

# Step 3: Parse clusters
clusters = parse_clstr_file(clstr_file)

# Step 4: Compute V-score
compute_v_score(clusters, dataset)

FileNotFoundError: [WinError 2] The system cannot find the file specified